<a href="https://colab.research.google.com/github/yepez26/BRFUNK_ismir_2024/blob/main/predict_topics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
import torch.nn as nn

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random
import pickle

import ast

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:

# Specify the path to the CSV file
excel_path = '/content/drive/MyDrive/UFRGS/mestrado/matitalk/analise_topicos_fabiola/funk_lyrics_fab.xlsx'

lyrics_test = pd.read_excel(excel_path)
lyrics_test['trecho'] = lyrics_test['text'].copy()

lyrics_test = lyrics_test[lyrics_test['trecho'].duplicated()==False].reset_index(drop=True)

path = '/content/drive/MyDrive/UFRGS/mestrado/matitalk/analise_topicos_fabiola/'

df = pd.read_excel(path+'classification_test_ex2.xlsx')

score = []
for i in range(0,len(df)):
  aux = df['score'][i]
  aux = ast.literal_eval(aux)['answer']
  score.append(aux)

df['score2'] = score
df['score2'] = np.where(df['score2'].str.contains('Sem'),0,df['score2'])
df['score'] = df['score2'].astype(int)


In [ ]:
#load model
from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads


tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 150,
                       pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )

In [ ]:
def print_rand_sentence_encoding():
  '''Displays tokens, token IDs and attention mask of a random text sample'''
  index = random.randint(0, len(text) - 1)
  tokens = tokenizer.tokenize(tokenizer.decode(token_id[index]))
  token_ids = [i.numpy() for i in token_id[index]]
  attention = [i.numpy() for i in attention_masks[index]]

  table = np.array([tokens, token_ids, attention]).T
  print(tabulate(table,
                 headers = ['Tokens', 'Token IDs', 'Attention Mask'],
                 tablefmt = 'fancy_grid'))

In [ ]:

def b_tp(preds, labels):
  '''Returns True Positives (TP): count of correct predictions of actual class 1'''
  return sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_fp(preds, labels):
  '''Returns False Positives (FP): count of wrong predictions of actual class 1'''
  return sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_tn(preds, labels):
  '''Returns True Negatives (TN): count of correct predictions of actual class 0'''
  return sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_fn(preds, labels):
  '''Returns False Negatives (FN): count of wrong predictions of actual class 0'''
  return sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_metrics(preds, labels):
  '''
  Returns the following metrics:
    - accuracy    = (TP + TN) / N
    - precision   = TP / (TP + FP)
    - recall      = TP / (TP + FN)
    - specificity = TN / (TN + FP)
  '''
  preds = np.argmax(preds, axis = 1).flatten()
  labels = labels.flatten()
  tp = b_tp(preds, labels)
  tn = b_tn(preds, labels)
  fp = b_fp(preds, labels)
  fn = b_fn(preds, labels)
  b_accuracy = (tp + tn) / len(labels)
  b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
  b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
  b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'
  return b_accuracy, b_precision, b_recall, b_specificity

In [ ]:

def softmax(array):

  # Assuming 'prediction' is the output logits
  logits = np.array(array)


  sigmoid = lambda x: 1 / (1 + np.exp(-x))
  probabilities = sigmoid(logits)
  # Apply softmax function
  # probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)

  # Extract the probability of the positive class (assuming it's the first class)
  return probabilities[0][0]


def modified_softmax(logits, alpha=1):
    exp_logits = np.exp(alpha * logits)
    max_exp = np.max(exp_logits)
    softmax_output = exp_logits / np.sum(exp_logits)
    return softmax_output * (1 - max_exp) + max_exp

def predict(new_sentence):

  # We need Token IDs and Attention Mask for inference on the new sentence
  test_ids = []
  test_attention_mask = []
  # Apply the tokenizer
  encoding = preprocessing(new_sentence, tokenizer)

  # Extract IDs and Attention Mask
  test_ids.append(encoding['input_ids'])
  test_attention_mask.append(encoding['attention_mask'])
  test_ids = torch.cat(test_ids, dim = 0)
  test_attention_mask = torch.cat(test_attention_mask, dim = 0)

  # Forward pass, calculate logit predictions
  with torch.no_grad():
    output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

  prediction =np.argmax(output.logits.cpu().numpy()).flatten().item()

  # prediction = np.max(modified_softmax(output.logits.cpu().numpy()))
  return prediction

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Specify the path to the CSV file
path = '/content/drive/MyDrive/UFRGS/mestrado/matitalk/analise_topicos_fabiola//'

excel_path = path+'funk_lyrics_fab.xlsx'

lyrics_test = pd.read_excel(excel_path)

lyrics_test = lyrics_test[lyrics_test['text'].duplicated()==False].reset_index(drop=True)

lyrics_test['trecho'] = lyrics_test['text'].copy()


In [ ]:
for topic in [0,1,2,3,4,5,6,7,9,11,12]:

  # Load the model from file
  with open("/content/drive/MyDrive/UFRGS/mestrado/bert_class_data_aug/models/model_topic_{}.pkl".format(topic), "rb") as f:
      model = pickle.load(f)
  predict_values = []
  for text in lyrics_test['trecho']:

      value = predict(text)
      predict_values.append(value)

      # print(text)
      # print(value)

  lyrics_test[topic] = predict_values

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input 

In [ ]:
lyrics_test.to_excel('/content/drive/MyDrive/UFRGS/mestrado/bert_class_data_aug/funk_lyrics_fab_topics.xlsx')